In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES = 2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
import pandas as pd
import torch.nn as nn

from benchmark import tracker

pd.set_option("display.precision", 2)

In [3]:
device = 'cuda:0'
device

'cuda:0'

In [4]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [5]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, memory_unit='KB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.4,0.07,0.45,0.09,33786.37,131.07,545259520.0,33280.0,1435.04
conv_64_512_3x3,0.77,0.04,1.26,0.02,36835.33,1179.65,4690151424.0,295424.0,733.52
bottleneck_64_512_3x3,0.69,0.04,1.51,0.03,37294.08,2686.98,2383218688.0,150048.0,787.19
conv_64_512_3x3_g2,1.89,0.04,2.66,0.02,34849.28,3211.26,11956733952.0,803328.0,1604.36
conv_64_512_3x3_g8,0.68,0.03,1.52,0.04,32983.04,802.82,2994898944.0,201216.0,840.11
conv_padding,0.77,0.05,1.02,0.06,36835.33,91357.18,4840226816.0,295424.0,737.46
conv_nn_Padding,0.85,0.04,1.33,0.02,41291.78,1179.65,4840226816.0,295424.0,792.63


In [6]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True, memory_unit='MB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.52,0.05,0.5,0.11,38.24,35.65,545259520.0,33280.0,469.11
conv_64_512_3x3,1.0,0.03,1.38,0.1,39.03,35.65,4690151424.0,295424.0,904.15
bottleneck_64_512_3x3,0.91,0.05,2.45,0.51,41.57,35.65,2383218688.0,150048.0,1293.05
conv_64_512_3x3_g2,1.89,0.03,13.54,0.49,39.24,41.11,11956733952.0,803328.0,5386.85
conv_64_512_3x3_g8,1.07,0.03,12.09,0.07,37.37,36.45,2994898944.0,201216.0,4582.76
conv_padding,0.98,0.03,2.09,0.05,40.04,35.65,4840226816.0,295424.0,1137.61
conv_nn_Padding,1.05,0.05,1.39,0.06,44.3,35.65,4840226816.0,295424.0,980.07


In [7]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, amp=True, memory_unit='GB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.47,0.05,0.63,0.14,0.04,0.04,545259520.0,33280.0,509.43
conv_64_512_3x3,0.58,0.05,0.71,0.27,0.04,0.04,4690151424.0,295424.0,563.59
bottleneck_64_512_3x3,0.84,0.04,1.3,0.09,0.04,0.04,2383218688.0,150048.0,908.77
conv_64_512_3x3_g2,10.62,0.07,12.07,0.04,0.04,0.04,11956733952.0,803328.0,7843.02
conv_64_512_3x3_g8,2.53,0.03,9.89,0.06,0.04,0.04,2994898944.0,201216.0,4316.39
conv_padding,0.58,0.05,0.74,0.07,0.04,0.04,4840226816.0,295424.0,568.43
conv_nn_Padding,0.65,0.06,0.72,0.07,0.04,0.04,4840226816.0,295424.0,625.85


In [8]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True, amp=True, memory_unit='B')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.42,0.03,0.5,0.1,37980672.0,35654144.0,545259520.0,33280.0,435.67
conv_64_512_3x3,0.54,0.03,0.71,0.06,38805504.0,35654144.0,4690151424.0,295424.0,541.81
bottleneck_64_512_3x3,0.81,0.05,1.29,0.09,39493120.0,35654144.0,2383218688.0,150048.0,888.35
conv_64_512_3x3_g2,10.62,0.07,12.07,0.04,38068736.0,41105920.0,11956733952.0,803328.0,7840.24
conv_64_512_3x3_g8,2.55,0.03,9.9,0.07,36202496.0,36454400.0,2994898944.0,201216.0,4323.25
conv_padding,0.58,0.05,0.71,0.08,39845888.0,35654144.0,4840226816.0,295424.0,560.45
conv_nn_Padding,0.66,0.07,0.74,0.09,44304384.0,35654144.0,4840226816.0,295424.0,635.28
